In [1]:
#%pip install -U fastmcp

<div class="alert alert-block alert-info" style="background-color: #05299E; color: white;">
    <h1 style="color: white;">LangGraph Agent Client</h1>
    <h4 style="color: white;">Cross-framework test for LangGraph Agents <<>> OpenAI Agents</h4>
    <h6 style="color: white;">@author: benjamin-chu@outlook.com</h6>
</div>

In [1]:
import asyncio
import json 
import nest_asyncio
import operator
import os
import sys
import traceback

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage, ToolMessage
from langchain_core.tools import BaseTool
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Annotated, Any, Dict, List, Tuple, TypedDict, Optional

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
AZURE_MCP_HHAI_ENDPOINT = os.getenv("AZURE_MCP_HHAI_ENDPOINT")
AZURE_MCP_HHAI_API_KEY = os.getenv("AZURE_MCP_HHAI_API_KEY")

In [3]:
# --- LLM Definitions ---
# LLM for the ReAct agent
react_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [4]:
nest_asyncio.apply()

In [6]:
# --- Agent State Definition ---
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    next_node: str
    original_query: str
    final_agent_response: str

# --- Graph Node Definitions ---
# Orchestrator Node
class RouteQuery(BaseModel):
    """Route the user's query to the appropriate service: appointments_service, info_service, or end if unclear."""
    destination: str = Field(description="The service to route to: 'appointments_service', 'info_service', or 'cannot_route'.")

async def route_query_node(state: AgentState):
    print("\n--- Router Node ---")
    user_message = state["messages"][-1].content
    print(f"Routing query: {user_message}")

    structured_llm = react_llm.with_structured_output(RouteQuery)
    
    prompt = f"""You are an expert query router. Based on the user's query, determine if it relates to:
    1. 'appointments_service': For making, modifying, viewing, or cancelling appointment bookings.
    2. 'info_service': For general information, like vaccine side effects or other health questions.
    If the query is unclear or doesn't fit these, choose 'cannot_route'.

    User query: "{user_message}"
    """
    try:
        routing_decision = await structured_llm.ainvoke(prompt)
        print(f"Routing decision: {routing_decision.destination}")
        
        if routing_decision.destination not in ["appointments_service", "info_service"]:
            final_dest = "cannot_route"
        else:
            final_dest = routing_decision.destination
        return {"next_node": final_dest, "original_query": user_message}
    except Exception as e:
        print(f"Error in router LLM: {e}")
        return {"next_node": "cannot_route", "original_query": user_message}


async def react_agent_node(state: AgentState, agent_executor: Any, node_name: str):
    print(f"\n--- {node_name} (Agent Node) ---")
    
    messages_for_react = [HumanMessage(content=state["original_query"])]
    print(f"Invoking agent for '{node_name}' with query: {state['original_query']}")

    try:
        response = await agent_executor.ainvoke({"messages": messages_for_react})
        
        ai_message_content = "No AIMessage content found from the agent."
        if response["messages"] and isinstance(response["messages"][-1], AIMessage):
            ai_message_content = response["messages"][-1].content
        
        print(f"Agent response for {node_name}: {ai_message_content}")
        return {
            "messages": state["messages"] + [AIMessage(content=ai_message_content)],
            "final_agent_response": ai_message_content,
            "next_node": "end_conversation"
        }
    except Exception as e:
        print(f"Error in the agent node {node_name}: {e}\n{traceback.format_exc()}")
        error_response = f"Sorry, an error occurred while processing your request via {node_name}."
        return {
            "messages": state["messages"] + [AIMessage(content=error_response)],
            "final_agent_response": error_response,
            "next_node": "end_conversation"
        }

# Conditional Edge Logic
def should_route_to_service(state: AgentState):
    print(f"\n--- Conditional Routing Decision ---")
    destination = state.get("next_node")
    print(f"Next node from state for conditional edge: {destination}")
    if destination == "appointments_service":
        return "appointments_node"
    elif destination == "info_service": 
        return "healthhub_info_node"
    else: # "cannot_route" or any other case
        if destination == "cannot_route":
            state["final_agent_response"] = "I'm sorry, I couldn't determine how to handle your request with the available services. Please try rephrasing."
            state["messages"] = state["messages"] + [AIMessage(content=state["final_agent_response"])]
        return END


async def run_client():
    print("Connecting to FastMCP server...")

    # --- Define MCP Server Connections ---
    mcp_server_configs = {
        "local_services": { # OpenAI agents local server
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        },
        "healthhub_remote": { # HealthHub AI server
            "url": AZURE_MCP_HHAI_ENDPOINT, 
            "headers": {"x-api-key": AZURE_MCP_HHAI_API_KEY},
            "transport": "sse",
            "cache_tools_list": True,
        }
    }
    
    async with MultiServerMCPClient(mcp_server_configs) as client:
        print("Connected to all servers successfully!")
        
        all_mcp_tools: List[BaseTool] = client.get_tools()
        
        if not all_mcp_tools:
            print("No tools found from any server. Check server definitions and connections.")
            return

        # --- Dynamically Filter Tools for Each Service by Servers ---
        local_appointments_tools: List[BaseTool] = []
        remote_healthhub_tools: List[BaseTool] = []

        for tool in all_mcp_tools:
            if tool.name.startswith("appointments_agent"): # Tools from your local server
                if "appointments_agent" in tool.name:
                     local_appointments_tools.append(tool)
            elif tool.name.startswith("healthhub_ai"): # Tools from the HealthHub AI server
                remote_healthhub_tools.append(tool)
        
        print("\n--- Filtered Tools ---")
        print(f"Local Appointments Service Tools ({len(local_appointments_tools)}): {[t.name for t in local_appointments_tools]}")
        if not local_appointments_tools: print("Warning: No tools found for Local Appointments Service.")
        
        print(f"Remote HealthHub Info Tools ({len(remote_healthhub_tools)}): {[t.name for t in remote_healthhub_tools]}")
        if not remote_healthhub_tools: print("Warning: No tools found for Remote HealthHub Info Service.")
        print("-" * 80 + "\n")

        # --- Create Specialised Agent Executors ---
        appointments_agent_executor = create_react_agent(react_llm, local_appointments_tools) if local_appointments_tools else None
        healthhub_info_agent_executor = create_react_agent(react_llm, remote_healthhub_tools) if remote_healthhub_tools else None

        if not appointments_agent_executor:
            print("Critical: Appointments agent executor (local) could not be created.")
        if not healthhub_info_agent_executor:
            print("Critical: HealthHub info agent executor (remote) could not be created.")

        # --- Build the LangGraph Workflow ---
        workflow = StateGraph(AgentState)

        workflow.add_node("router", route_query_node)

        async def appointments_node_wrapper(state: AgentState):
            if not appointments_agent_executor:
                return {"messages": state["messages"] + [AIMessage(content="Local Appointments service is unavailable.")], "final_agent_response": "Local Appointments service unavailable.", "next_node": "end_conversation"}
            return await react_agent_node(state, appointments_agent_executor, "Local Appointments Service Node")
        
        # New node wrapper for the remote HealthHub service
        async def healthhub_info_node_wrapper(state: AgentState):
            if not healthhub_info_agent_executor:
                return {"messages": state["messages"] + [AIMessage(content="Remote HealthHub Info service is unavailable.")], "final_agent_response": "Remote HealthHub Info service unavailable.", "next_node": "end_conversation"}
            return await react_agent_node(state, healthhub_info_agent_executor, "Remote HealthHub Info Service Node")

        workflow.add_node("appointments_node", appointments_node_wrapper)
        workflow.add_node("healthhub_info_node", healthhub_info_node_wrapper) # Add the new node

        workflow.set_entry_point("router")
        workflow.add_conditional_edges("router", should_route_to_service, {
            "appointments_node": "appointments_node", 
            "healthhub_info_node": "healthhub_info_node",
            END: END
        })
        workflow.add_edge("appointments_node", END)
        workflow.add_edge("healthhub_info_node", END) 

        app = workflow.compile()

        # --- Interactive Chat ---
        print("\n--- Starting Interactive Chat ---")
        print("Connected to Local Appointments and Remote HealthHub Info services.")
        print("Type 'done', 'exit' or 'quit' to end the chat.")
        
        while True:
            try:
                user_input = input("User: ")
                if user_input.lower() in ["done", "exit", "quit"]:
                    print("Exiting conversation chat.")
                    break
                if not user_input.strip():
                    continue
                
                initial_state = AgentState(
                    messages=[HumanMessage(content=user_input)],
                    next_node="", original_query="", final_agent_response="" 
                )
                
                print("Processing your request...")
                final_state_dict = await app.ainvoke(initial_state)
                final_state = AgentState(**final_state_dict)

                print("\n--- Assistant Response ---")
                final_response_content = final_state.get('final_agent_response', "Sorry, I couldn't process that.")
                print(f"Assistant: {final_response_content}")
                print("-" * 80)

            except KeyboardInterrupt:
                print("\nExiting conversation chat due to interrupt.")
                break
            except Exception as e:
                print(f"An error occurred during conversation chat: {e}")
                traceback.print_exc()

In [7]:
# Run the client
print("Run the client >>")
asyncio.run(run_client())

Run the client >>
Connecting to FastMCP server...
Connected to all servers successfully!

--- Filtered Tools ---
Local Appointments Service Tools (1): ['appointments_agent_process_booking_request']
Remote HealthHub Info Tools (1): ['healthhub_ai_tool']
--------------------------------------------------------------------------------


--- Starting Interactive Chat ---
Connected to Local Appointments and Remote HealthHub Info services.
Type 'exit' or 'quit' to end the chat.


User:  my user id is 123. what appointments i have so far?


Processing your request...

--- Router Node ---
Routing query: my user id is 123. what appointments i have so far?
Routing decision: appointments_service

--- Conditional Routing Decision ---
Next node from state for conditional edge: appointments_service

--- Local Appointments Service Node (Agent Node) ---
Invoking agent for 'Local Appointments Service Node' with query: my user id is 123. what appointments i have so far?
Agent response for Local Appointments Service Node: You have an upcoming booking for the Influenza (INF) vaccine at Polyclinic A on March 1, 2025, at 10:00 AM. 

If you need any changes or further assistance, let me know!

--- Assistant Response ---
Assistant: You have an upcoming booking for the Influenza (INF) vaccine at Polyclinic A on March 1, 2025, at 10:00 AM. 

If you need any changes or further assistance, let me know!
--------------------------------------------------------------------------------


User:  what are the side effects of influenza vaccine


Processing your request...

--- Router Node ---
Routing query: what are the side effects of influenza vaccine
Routing decision: info_service

--- Conditional Routing Decision ---
Next node from state for conditional edge: info_service

--- Remote HealthHub Info Service Node (Agent Node) ---
Invoking agent for 'Remote HealthHub Info Service Node' with query: what are the side effects of influenza vaccine
Agent response for Remote HealthHub Info Service Node: The side effects of the influenza vaccine are generally mild and may include:

1. Soreness, redness, or swelling at the injection site.
2. Low-grade fever.
3. Headache and muscle aches.
4. Runny nose, cough, or sore throat.
5. Vomiting (more common in children).

Severe allergic reactions are rare but can occur, requiring immediate medical attention. If you have any concerns, it's best to consult your healthcare professional.

Would you like to know how to manage any side effects you might experience after getting the flu vaccine?



User:  exit


Exiting chat.
